## Prerequisites

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import numpy as np
import pandas as pd

from src.dataset.data import Dataset
from src.features.build_features import *
from src.features.process_fold import *
from src.model.train import *

import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_rows = 100
pd.options.display.max_columns = None

%matplotlib inline

In [4]:
ds = Dataset()
ds.load_dataset("30000")

In [5]:
build_processed_dataset(ds)

In [6]:
X = ds.X_train.reset_index()
y = ds.y_train.reset_index()['isFraud']
X_test = ds.X_test.reset_index()
X['isFraud'] = y

## Sandbox

In [7]:
X.head(10)

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,has_identity,lastest_browser,screen_width,screen_height,screen_resolution,id_31_device,is_win8_vista,is_windows_otheros,id_30_os,id_30_version,device_name,device_version,deviceInfo_device,deviceInfo_version,uid,uid2,uid3,uid4,uid5,bank_type,DT,DT_M,DT_W,DT_D,DT_hour,DT_day_week,DT_day_month,DT_weekday,DT_weekend,DT_night,DT_morning,DT_afternoon,DT_evening,sinus_hour,cos_hour,local_time,local_time_M,local_time_W,local_time_D,local_time_hour,local_time_day_week,local_time_day,local_time_weekday,local_time_weekend,local_time_night,local_time_morning,local_time_afternoon,local_time_evening,local_time_sinus_hour,local_time_cos_hour,is_december,is_holiday,TransactionAmt_main,TransactionAmt_cents,TransactionAmt_n_decimals,W_0_cents,W_50_cents,W_95_cents,TransactionAmt_Log,product_type,TransactionAmt_check,is_proton_mail,email_check,P_emaildomain_bin,P_emaildomain_suffix,R_emaildomain_bin,R_emaildomain_suffix,M_sum,M_na,D9_not_na,D8_not_same_day,D8_D9_decimal_dist,D1_scaled,D2_scaled,groupby_card1_TransactionAmt_mean,groupby_card1_TransactionAmt_std,groupby_card1_TransactionAmt_std_norm,groupby_card4_TransactionAmt_mean,groupby_card4_TransactionAmt_std,groupby_card4_TransactionAmt_std_norm,groupby_uid_TransactionAmt_mean,groupby_uid_TransactionAmt_std,groupby_uid_TransactionAmt_std_norm,groupby_uid2_TransactionAmt_mean,groupby_uid2_TransactionAmt_std,groupby_uid2_TransactionAmt_std_norm,groupby_uid3_TransactionAmt_mean,groupby_uid3_TransactionAmt_std,groupby_uid3_TransactionAmt_std_norm,groupby_uid4_TransactionAmt_mean,groupby_uid4_TransactionAmt_std,groupby_uid4_TransactionAmt_std_norm,groupby_uid5_TransactionAmt_mean,groupby_uid5_TransactionAmt_std,groupby_uid5_TransactionAmt_std_norm,groupby_bank_type_TransactionAmt_mean,groupby_bank_type_TransactionAmt_std,groupby_bank_type_TransactionAmt_std_norm,groupby_addr1_TransactionAmt_mean,groupby_addr1_TransactionAmt_std,grou

In [8]:
from sklearn.model_selection import KFold
folds = KFold(n_splits=5, random_state=0, shuffle=False)

In [9]:
train_index, valid_index = list(folds.split(X))[0]
X_train, X_valid = (X.iloc[train_index], X.iloc[valid_index])
y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

In [10]:
fold = Fold(X_train, y_train, X_valid, y_valid, X_test, ds.categorical_cols)

In [11]:
process_fold(fold)

In [12]:
fold.X_train.head(5)

,TransactionID,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_32,id_34,id_35,id_36,id_37,id_38,DeviceType,has_identity,lastest_browser,screen_width,screen_height,screen_resolution,id_31_device,is_win8_vista,is_windows_otheros,id_30_os,id_30_version,device_name,device_version,deviceInfo_device,deviceInfo_version,DT_hour,DT_day_week,DT_day_month,DT_weekday,DT_weekend,DT_night,DT_morning,DT_afternoon,DT_evening,sinus_hour,cos_hour,local_time_hour,local_time_day_week,local_time_day,local_time_weekday,local_time_weekend,local_time_night,local_time_morning,local_time_afternoon,local_time_evening,local_time_sinus_hour,local_time_cos_hour,is_december,is_holiday,TransactionAmt_main,TransactionAmt_cents,TransactionAmt_n_decimals,W_0_cents,W_50_cents,W_95_cents,TransactionAmt_Log,product_type,TransactionAmt_check,is_proton_mail,email_check,P_emaildomain_bin,P_emaildomain_suffix,R_emaildomain_bin,R_emaildomain_suffix,M_sum,M_na,D9_not_na,D8_not_same_day,D8_D9_decimal_dist,D1_scaled,D2_scaled,groupby_card1_TransactionAmt_mean,groupby_card1_TransactionAmt_std,groupby_card1_TransactionAmt_std_norm,groupby_card4_TransactionAmt_mean,groupby_card4_TransactionAmt_std,groupby_card4_TransactionAmt_std_norm,groupby_uid_TransactionAmt_mean,groupby_uid_TransactionAmt_std,groupby_uid_TransactionAmt_std_norm,groupby_uid2_TransactionAmt_mean,groupby_uid2_TransactionAmt_std,groupby_uid2_TransactionAmt_std_norm,groupby_uid3_TransactionAmt_mean,groupby_uid3_TransactionAmt_std,groupby_uid3_TransactionAmt_std_norm,groupby_uid4_TransactionAmt_mean,groupby_uid4_TransactionAmt_std,groupby_uid4_TransactionAmt_std_norm,groupby_uid5_TransactionAmt_mean,groupby_uid5_TransactionAmt_std,groupby_uid5_TransactionAmt_std_norm,groupby_bank_type_TransactionAmt_mean,groupby_bank_type_TransactionAmt_std,groupby_bank_type_TransactionAmt_std_norm,groupby_addr1_TransactionAmt_mean,groupby_addr1_TransactionAmt_std,groupby_addr1_TransactionAmt_std_norm,groupby_card1_id_02_mean,groupby_card1_id_02_std,groupby_card1_id_02_std_norm,groupby_card4_id_02_mean,groupby_card4_id_02_std,groupby_card4_id_02_std_norm,groupby_uid_id_02_mean,groupby_uid_id_02_std,groupby_uid_id_02_std_norm,groupby_uid2_id_02_mean,groupby_uid2_id_02_std,groupby_uid2_id_02_std_norm,groupby_uid3_id_02_mean,groupby_uid3_id_02_std,groupby_uid3_id_02_std_norm,groupby_uid4_id_02_mean,groupby_uid4_id_02_std,groupby_uid4_id_02_std_norm,groupby_uid5_id_02_mean,groupby_uid5_id_02_std,groupby_uid5_id_02_std_norm,groupby_bank_type_id_02_mean,groupby_bank_type_id_02_std,groupby_bank_type_id_02_std_norm,groupby_addr1_id_02_mean,groupby_addr1_id_02_std,groupby_addr1_id_02_std_norm,groupby_card1_D15_mean,groupby_card1_D15_std,groupby_card1_D15_std_norm,groupby_card4_D15_mean,groupby_card4_D15_std,groupby_card4_D15_std_norm,groupby_uid_D15_mean,groupby_uid_D15_std,groupby_uid_D15_std_norm,groupby_uid2_D15_mean,groupby_uid2_D15_std,groupby_uid2_D15_std_norm,groupby_uid3_D15_mean,groupby_uid3_D15_std,groupby_uid3_D15_std_norm,groupby_uid4_D15_mean,groupby_uid4_D15_std,groupby_uid4_D15_std_norm,groupby_uid5_D15_mean,groupby_uid5_D15_std,groupby_uid5_D15_std_norm,groupby_bank_type_D15_mean,groupby_bank_type_D15_std,groupby_bank_type_D15_std_norm,groupby_addr1_D15_mean,groupby_addr1_D15_std,groupby_addr1_D15_std_norm,id_02__id_20,id_02__D8,D11__DeviceInfo,DeviceInfo__P_emaildomain,P_emaildomain__C2,card2__dist1,card1__card5,card2__id_20,card5__P_emaildomain,addr1__card1,C11__C13,card3_DT_D_hour_dist,card3_DT_W_week_day_dist,card3_DT_M_month_day_dist,card3_DT_D_hour_dist_best,card3_DT_W_week_day_dist_best,card3_DT_M_month_day_dist_best,card5_DT_D_hour_dist,card5_DT_W_week